# Initialisation du notebook

Chargement des packages python

In [ ]:
import typing
from pathlib import Path

import pandas as pd

pd.set_option("display.max_rows", 200)
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

import bokeh
import bokeh.io

bokeh.io.output_notebook()


Réglage du formatage des figures matplolib.

In [ ]:
import matplotlib

matplotlib.rcdefaults()
matplotlib.rc(
    "axes",
    labelsize="large",
    labelweight="bold",
    titlesize="xx-large",
    titleweight="bold",
)


Chargement du jeu données OpenFoodFact et initialisation des variables globales.

In [ ]:
try:
    print(data_raw.shape)
except NameError:
    data_raw = pd.read_csv("fr.openfoodfacts.org.products.csv", sep="\t")
    print(data_raw.shape)


In [ ]:
from dataclasses import dataclass


@dataclass
class GlobalState:
    data_raw = None
    data = None


g = GlobalState()
g.data_raw = data_raw
g.data = data_raw


Définitions de fonctions utilisées tout au long de ce notebook.

In [ ]:
def feature_summary(data):
    col_names = list(data.columns)
    dtypes = [data[col].dtype.name for col in data]
    types = [type(data[col].iloc[0]).__name__ for col in data]
    nb_unique_val = [
        f"{len(data[col].unique())}"
        if isinstance(data[col].iloc[0], typing.Hashable)
        else "_"
        for col in data
    ]
    ratio_null = [data[col].isnull().sum() / data[col].shape[0] * 100 for col in data]

    table = pd.DataFrame(
        {
            "columns": col_names,
            "DType": dtypes,
            "Type": types,
            "Nb unique values": nb_unique_val,
            "% null": ratio_null,
        }
    ).set_index("columns")
    table = table.style.background_gradient(
        axis=None, vmin=0, vmax=np.max(ratio_null), cmap="Reds", subset=["% null"]
    )
    return table


def drop_null_features(data, max_ratio_null=0.8):
    valid_columns = [
        col for col in data if data[col].count() / data.shape[0] > (1 - max_ratio_null)
    ]
    return data[valid_columns]


def drop_features(data, features):
    subset_columns = [col for col in data if col not in features]
    return data[subset_columns]


def add_feature_in_france(data):
    data["in_france"] = ["france" in str(v) for v in data["countries_tags"]]
    return data


def only_products_from_france(data):
    new_data = data[data["in_france"]]
    return drop_features(new_data, ["in_france"])


def count_nb_nan_per_row(data):
    return data.isnull().sum(axis=1)


def remove_products_with_too_much_nan(data, nb_nans_max):
    return data[count_nb_nan_per_row(data) <= nb_nans_max]


def get_numerical_features(data):
    return [col for col in data if pd.api.types.is_numeric_dtype(data[col])]


def get_categorical_features(data):
    return [col for col in data if not pd.api.types.is_numeric_dtype(data[col])]


def get_category_unique(column):
    values = [vv for v in column for vv in str(v).split(",")]
    return list(np.unique(values, return_counts=True))


class CompareFeature:
    def __init__(self, data, features):
        self.data = data[features]
        self.features = features
        for i, f in enumerate(features):
            setattr(self, f"f{i}", f)

    def diff_nan(self):
        nb_nan = np.sum(pd.isnull(self.data), axis=1)
        return self.data[(nb_nan > 0) & (nb_nan < len(self.features))]


# Exploration et nettoyage des données

Regardons le dataset OpenFoodFacts.

In [ ]:
def dataset_summary(data, title):
    print(title)
    print(f"Nombre de produits : {data.shape[0]}")
    print(f"Nombre de variables : {data.shape[1]}")
    return data.sample(10)


dataset_summary(g.data_raw, "Dataset originel:")


## Produits vendus en France

Regardons le ratio du nombre de produits vendus par pays. Nous affichons uniquement les 5 premiers pays où il y a le plus de produits vendus. Un produit peut être vendu dans plusieurs pays et peut donc être comptabilisé dans les ratios de plusieurs pays. 

In [ ]:
unique_values, value_counts = get_category_unique(g.data_raw["countries_fr"])
df = (
    pd.DataFrame({"country": unique_values, "ratio": value_counts / len(g.data_raw)})
    .sort_values(by="ratio", ascending=False)
    .head(10)
)

df.plot.barh(
    x="country",
    y="ratio",
    color=["blue" if country != "France" else "red" for country in df["country"]],
)
plt.gca().get_legend().remove()
plt.xlabel("Ratio des produits")
_ = plt.ylabel("Pays")


L'application, qui sera développée, ne visera que le marché français dans un premier temps. Nous éliminons donc les produits qui ne sont pas vendus en France.

In [ ]:
add_feature_in_france(g.data_raw)

g.data = only_products_from_france(g.data_raw)

dataset_summary(g.data_raw, "Dataset originel:")
print()
dataset_summary(g.data, "Dataset nettoyé (en cours):")


## Nettoyage des variables

Nous retirons les variables qui n'ont pas d'intérêt pour la prédiction du Nutri-Score.

In [ ]:
g.data = drop_features(
    g.data,
    [
        # "code",
        "url",
        # "product_name",
        "image_url",
        "image_small_url",
        "creator",
        "created_datetime",
        # "created_t",
        "last_modified_t",
        "last_modified_datetime",
        "packaging",
        "packaging_tags",
        "countries",
        "countries_fr",
        "countries_tags",
        "states",
        "states_tags",
        "states_fr",
        "quantity",
        "purchase_places",
        # "ingredients_text",
        "categories",
        "categories_tags",
        "main_category",
        "brands_tags",
        "nutrition-score-uk_100g",
    ],
)


Regardons le taux de remplissage par variables.

In [ ]:
# feature_summary(g.data)
plt.figure(figsize=(10, 15), dpi=100)
(~g.data.isnull()).mean().sort_values().plot.barh(width=0.8)
plt.yticks(fontsize=5)
plt.axvline(x=0.45, color="red")
plt.xlabel("Taux de remplissage par variable")
plt.ylabel("Variables")
_ = plt.xlim(0, 1)


Nous enlevons les variables avec trop de valeurs manquantes, ici toutes les variables avec un taux de remplissage inferieur à 45%.

## Netoyage des produits selon leur taux de remplissage

In [ ]:
g.data = drop_null_features(g.data, max_ratio_null=0.55)
dataset_summary(g.data_raw, "Dataset originel:")
print()
dataset_summary(g.data, "Dataset nettoyé (en cours):")
feature_summary(g.data)


Intéressons nous, maintenant, aux valeurs manquant par produit.

In [ ]:
(
    g.data.isnull().sum(axis=1).value_counts().sort_index().cumsum() / g.data.shape[0]
).plot.bar()
plt.axvline(15.5, color="red")
plt.xlabel("Nb de valeurs manquantes par produit")
plt.ylabel("Fraction cummulative\ndu nombre total de produits")
_ = plt.ylim(0, 1)


Nous éliminons les produits avec trop de valeurs manquants. En effet, de telles produits sont si peu remplis qu'ils sont impossibles à traiter. Nous fixons ici la limite à un nombre limite de valeurs manquantes à 16, ce qui exclut environ 25% des produits.

In [ ]:
g.data = remove_products_with_too_much_nan(g.data, nb_nans_max=15)


In [ ]:
dataset_summary(g.data_raw, "Dataset originel:")
dataset_summary(g.data, "Dataset nettoyé (en cours):")
feature_summary(g.data)


In [ ]:
g.data.sample(10)


## Exploration du Nutri-Score

In [ ]:
g.data[["nutrition_grade_fr", "nutrition-score-fr_100g"]].sample(10)


Nous convertissons la variable "nutrition_grade_fr" (Nutri-Score) dans un format catégoriel ordonné et formatés en majuscule.

In [ ]:
g.data["nutrition_grade_fr"] = pd.Categorical(
    g.data.nutrition_grade_fr, ordered=True
).rename_categories(list("ABCDE"))

# g.data["nutrition_grade_fr"] = (
#     g.data["nutrition_grade_fr"].cat.add_categories(["NaN"]).fillna("NaN")
# )


## Exploration des variables "catégories"

In [ ]:
features_category = [
    "main_category_fr",
    "categories_fr",
    "pnns_groups_1",
    "pnns_groups_2",
]
g.data[features_category].sample(10)


In [ ]:
import re
import unicodedata

regex_bad_char = re.compile(r"[^a-zA-Z\s]")
regex_space = re.compile(r"\s+")
regex_tag_lang = re.compile(r"[a-z]{2}:")


def normalize_str(string: str, word_sep=" "):
    if pd.isna(string):
        return pd.NA
    string = str(string)
    string = "".join(
        c
        for c in unicodedata.normalize("NFD", string)
        if unicodedata.category(c) != "Mn"
    )
    string = string.lower()
    string = regex_tag_lang.sub(word_sep, string)
    string = regex_bad_char.sub(word_sep, string)
    string = regex_space.sub(word_sep, string)
    string = string.strip()
    if len(string) == 0:
        string = pd.NA
    return string


def categorize(data):
    categories_list = [ll for l in data.str.split(",").dropna() for ll in l]
    categories_list = [normalize_str(s) for s in categories_list]
    categories = np.unique(categories_list)
    return categories.tolist()


### PNNS_groups_1 & PNNS_groups_2

PNNS = Programme national nutrition santé 

In [ ]:
g.data["pnns_groups_1"] = pd.Categorical(
    g.data["pnns_groups_1"].apply(lambda s: normalize_str(s, word_sep="_")),
    ordered=False,
)
print(
    f"Nombre de catégories: {g.data['pnns_groups_1'].cat.categories.size} (sans \"nan\")"
)
g.data["pnns_groups_1"].cat.add_categories(["nan"]).fillna(
    "nan"
).value_counts().plot.barh()
plt.ylabel("PNNS groups 1")
_ = plt.xlabel("Nombre de produits")


La variable "pnns_groups_1" est composée de seulement 10 catégories.

In [ ]:
plt.figure(figsize=(6, 8))
g.data["pnns_groups_2"] = pd.Categorical(
    g.data["pnns_groups_2"].apply(lambda s: normalize_str(s, word_sep="_")),
    ordered=False,
)
print(
    f"Nombre de catégories: {g.data['pnns_groups_2'].cat.categories.size} (sans \"nan\")"
)
g.data["pnns_groups_2"].cat.add_categories(["nan"]).fillna(
    "nan"
).value_counts().plot.barh()
plt.ylabel("PNNS groups 2")
_ = plt.xlabel("Nombre de produits")


La variable "pnns_groups_2" est composée de seulement 36 catégories.

Nous allons maintenant regarder la relation entre les variable "pnns_groups_1" et "pnns_groups_2" en regardant les comptage de produits croisé selon les deux types de catégories.

In [ ]:
import matplotlib.colors


def plot_pnns_groups_1_vs_2():
    df = g.data[["pnns_groups_1", "pnns_groups_2"]].copy()
    for col in df:
        df[col] = df[col].cat.add_categories(["nan"]).fillna("nan")
    df = df.groupby(["pnns_groups_2", "pnns_groups_1"]).size().unstack()
    df = df.sort_values(list(df.columns), ascending=False)
    plt.figure(figsize=(14, 10))
    sns.heatmap(
        df,
        linewidth=0.1,
        annot=True,
        fmt="d",
        linecolor="lightgray",
        cmap="Reds",
        vmin=-100,
        norm=matplotlib.colors.LogNorm(),
    )
    _ = plt.xticks(rotation=45, ha="right")


plot_pnns_groups_1_vs_2()


Nous remarquons que :
- "pnns_groups_2" est une sous-catégories de "pnns_groups_1",
- les produits avec pour sous-catégories "alcoholic_beverages" et "tripe_dishes" n'ont pas de catégorie "pnns_groups_1" (valeur manquante),
- les catégorie et sous-catégorie "unknown" sont complétement corrélées.

Regardons maintenant les produits classées comme "unknown".

In [ ]:
g.data["product_name"][g.data["pnns_groups_1"] == "unknown"].sample(10)


Nous remarquons que certains produits aurait pu être labélisés selon une catégorie extante de "pnns_groups_1" et "pnns_groups_2". Les catégories "unknown" peuvent être interprété comme des valeurs manquantes.

Selon nos observations, nous allons associés les produits avec les sous-catégories "alcoholic_beverages" (boissons alcolisées) et "tripe_dishes" (plats à base de tripes) respectivement aux catégories "beverages" et "fish_meat_egg" de "pnns_groups_1".

Les catégorie et sous-catégories "unknown" seront remplacées par des valeurs à nul pour être remplies plus tard.

In [ ]:
mask_nulls_1 = g.data["pnns_groups_1"].isnull()
mask_alcohols = g.data["pnns_groups_2"] == "alcoholic_beverages"
mask_tripes = g.data["pnns_groups_2"] == "tripe_dishes"
mask_unknown = g.data["pnns_groups_1"] == "unknown"

g.data.loc[mask_alcohols, "pnns_groups_1"] = "beverages"
g.data.loc[mask_tripes, "pnns_groups_1"] = "fish_meat_eggs"
g.data.loc[mask_unknown, "pnns_groups_1"] = np.nan
g.data.loc[mask_unknown, "pnns_groups_2"] = np.nan

g.data["pnns_groups_1"] = g.data["pnns_groups_1"].cat.remove_unused_categories()
g.data["pnns_groups_2"] = g.data["pnns_groups_2"].cat.remove_unused_categories()

plot_pnns_groups_1_vs_2()


### Exploration de main_category_fr & categories_fr

In [ ]:
plt.figure(figsize=(6, 9))
column_cat = pd.Categorical(
    g.data["main_category_fr"].apply(normalize_str), ordered=False
)
print(f"Nb catégories: {len(column_cat.unique())}")
column_cat.value_counts().sort_values(ascending=False)[1:50].plot.barh()
plt.xlabel("Nombre de produits")
_ = plt.ylabel("main_category_fr")


In [ ]:
plt.figure(figsize=(6, 9))
column_cat = pd.Categorical(
    g.data["categories_fr"].apply(normalize_str).explode(),
    ordered=False,
)

print(f"Nb catégories: {len(column_cat.unique())}")
column_cat.value_counts().sort_values(ascending=False)[1:50].plot.barh()
plt.xlabel("Nombre de produits")
_ = plt.ylabel("categories_fr")


#### Embeddings des intitulées de catégories principales

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
try:
    # https://docs.rapids.ai/api/cuml/stable/api.html#tsne
    from cuml.manifold import TSNE
except Exception:
    from sklearn.manifold import TSNE
from umap import UMAP


def text_feature_to_2d_coord(data, feature_name, tfidf_kwargs=None, tsne_kwargs=None, nb_samples=None, use_umap=False, force_compute=False):

    df = (
        data[feature_name]
        .apply(lambda t: normalize_str(t))
        .dropna()
    )

    if tfidf_kwargs is None:
        tfidf_kwargs = dict()

    text_transformer = TfidfVectorizer(**tfidf_kwargs)
    text_transformer.fit_transform(df)
    print("Nb of words used:", len(text_transformer.get_feature_names_out()))
    print("Nb of words excluded:", len(text_transformer.stop_words_))
    print(list(text_transformer.get_feature_names_out())[:10])
    print(list(sorted(text_transformer.stop_words_, key=lambda s: len(s))[:10]) + list(text_transformer.stop_words_)[:10])
    
    if tsne_kwargs is None:
        tsne_kwargs = dict()

    if use_umap:
        umap_kwargs_ = dict(
            n_components=2,
            n_neighbors=50,
            #force_approximation_algorithm=True,
            init="spectral",
            min_dist=5,
            spread=5,
            metric="hellinger",
            verbose=True,
            n_epochs=1000,
        )
        umap_kwargs_.update(tsne_kwargs)
        reducer = UMAP(**umap_kwargs_)
    else:
        tsne_kwargs_ = dict(
            n_components=2,
            perplexity=50,
            #perplexity=200,
            init="random",
            n_iter=10000,
            n_iter_without_progress=1000,
            # n_neighbors=200,
            verbose=True,
        )
        tsne_kwargs_.update(tsne_kwargs)
            
        reducer= TSNE(**tsne_kwargs_)


    file = Path(f"tsne_coord_{feature_name}.npy")
    
    use_cache = not force_compute
    if use_cache and file.exists() and nb_samples is None:
        coord_2d = np.load(file)
    else:
        if nb_samples:
            df = df.sample(nb_samples)
            
        text_embedding = text_transformer.transform(df)
        mask_non_null_embedding = np.array((text_embedding.sum(axis=1) != 0)).reshape(-1)

        coord_2d = np.full(fill_value=np.nan, shape=(df.shape[0], 2))
        te = text_embedding[mask_non_null_embedding]
        coord_2d[mask_non_null_embedding] = reducer.fit_transform(te)

        np.save(file, coord_2d)

    data[f"{feature_name}_emb_x"] = pd.Series(np.nan)
    data.loc[df.index, f"{feature_name}_emb_x"] = coord_2d[:, 0]

    data[f"{feature_name}_emb_y"] = pd.Series(np.nan)
    data.loc[df.index, f"{feature_name}_emb_y"] = coord_2d[:, 1]

    

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import Legend, ColumnDataSource, HoverTool
from itertools import cycle


def style_color_marker(n, color_list, marker_list):
    style_list = []
    all_styles = cycle(
        [(color, marker) for marker in marker_list for color in color_list]
    )
    return [next(all_styles) for _ in range(n)]


def plot_tsne_with_bokeh(
    data, tsne_var, group_cat, tooltip_var=["product_name", "nutrition_grade_fr"]
):
    data = pd.DataFrame(data)

    cat_column = (
        data[group_cat].astype("category").cat.add_categories(["NaN"]).fillna("NaN")
    )
    categories = cat_column.cat.categories

    p = figure(
        height=600,
        width=900 + len(categories) // 19 * 230,
        tools="pan,wheel_zoom,reset",
    )
    try:
        p.add_layout(Legend(ncols=2, nrows=19), "right")  # bokeh 3.X
    except Exception:
        p.add_layout(Legend())  # bokeh 2.X

    colors = sns.color_palette("tab10").as_hex()
    markers = [
        "circle",
        "triangle",
        "square",
        "plus",
        "star",
        "square_pin",
        "inverted_triangle",
        "diamond",
        "hex",
    ]

    renderer_list = []
    for (
        group,
        (color, marker),
    ) in zip(categories, style_color_marker(len(categories), colors, markers)):
        mask = cat_column[data.index] == group
        df = data[mask]
        data_source = ColumnDataSource(df)

        glyph = p.scatter(
            source=data_source,
            x=f"{tsne_var}_emb_x",
            y=f"{tsne_var}_emb_y",
            legend_label=str(group),
            fill_alpha=0.4,
            size=8,
            line_width=0.2,
            line_color="white",
            color=color,
            marker=marker,
        )
        renderer_list.append(glyph)
        # glyph.muted = True

    hover_tool = HoverTool(
        renderers=renderer_list,
        tooltips=[(var, f"@{var}") for var in tooltip_var],
    )
    p.add_tools(hover_tool)
    p.legend.click_policy = "hide"

    p.xaxis.axis_label = "Embedding X"
    p.xaxis.axis_label_text_font_style = "bold"
    p.yaxis.axis_label = "Embedding Y"
    p.yaxis.axis_label_text_font_style = "bold"

    return p


def plot_tsne_with_matplotlib(data, tsne_var, group_cat):
    data,
    tsne_var,
    group_cat,
    interactive_plot = (False,)
    tooltip_var = (["product_name", "nutrition_grade_fr"],)

    cat_column = data[group_cat].astype(
        "category"
    )  # .cat.add_categories(["NaN"]).fillna("NaN")
    categories = cat_column.cat.categories

    colors = sns.color_palette("tab10").as_hex()
    markers = [
        ".",
        "o",
        "v",
        "^",
        "<",
        ">",
        "8",
        "s",
        "p",
        "*",
        "h",
        "H",
        "D",
        "d",
        "P",
        "X",
    ]

    plt.figure(figsize=(12, 12))
    for (
        group,
        (color, marker),
    ) in zip(categories, style_color_marker(len(categories), colors, markers)):
        mask = cat_column[data.index] == group
        df = data[mask]
        plt.scatter(
            x=df[f"{tsne_var}_emb_x"],
            y=df[f"{tsne_var}_emb_y"],
            label=str(group),
            c=color,
            marker=marker,
            alpha=0.2,
            lw=0,
        )
    plt.xlabel("Embedding X")
    plt.ylabel("Embedding Y")
    leg = plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
    for lh in leg.legendHandles:
        lh.set_alpha(1)
        lh.set_sizes([120])
    return plt.gca()


def plot_tsne(
    data,
    tsne_var,
    group_cat,
    interactive_plot=False,
    tooltip_var=["product_name", "nutrition_grade_fr"],
    plot=True,
):
    if interactive_plot:
        plot = plot_tsne_with_bokeh(data, tsne_var, group_cat, tooltip_var)
        show(plot)
        return plot
    else:
        return plot_tsne_with_matplotlib(data, tsne_var, group_cat)


In [ ]:
variable = "main_category_fr"
df = pd.DataFrame(
    {"main_category": g.data[variable].apply(normalize_str).fillna("NA").unique()}
)
df_counts = g.data[variable].apply(normalize_str).fillna("NA").value_counts()
df["counts"] = [df_counts[v] for v in df["main_category"]]


threshold = (1, 10, 50, 100, 500, 1000, 1500)
condlist = [df["counts"] <= n for n in threshold]
choicelist = [f"n <= {n}" for n in threshold] + [f"n > {threshold[-1]}"]
df["cat"] = pd.Categorical(
    np.select(condlist, choicelist[:-1], default="inf"), categories=choicelist
)

text_feature_to_2d_coord(
    data=df,
    feature_name="main_category",
    tfidf_kwargs=dict(min_df=2, max_df=0.2),
    tsne_kwargs=dict(perplexity=100, n_iter=2000),
    # use_umap=True,
    # tsne_kwargs=dict(n_neighbors=10, min_dist=0.1, spread=0.1),
    # force_compute=True,
)
plot = plot_tsne(
    df,
    "main_category",
    group_cat="cat",
    tooltip_var=["main_category", "counts", "cat"],
    interactive_plot=False,
)


In [ ]:
df = df.set_index("main_category")

g.data["main_category_fr_emb_x"] = [
    df.loc[cat]["main_category_emb_x"]
    for cat in g.data["main_category_fr"].apply(normalize_str).fillna("NA")
]
g.data["main_category_fr_emb_y"] = [
    df.loc[cat]["main_category_emb_y"]
    for cat in g.data["main_category_fr"].apply(normalize_str).fillna("NA")
]


In [ ]:
# plot = plot_tsne(
#     g.data,
#     "main_category_fr",
#     group_cat="pnns_groups_1",
#     tooltip_var=["product_name", "nutrition_grade_fr", "main_category_fr"],
# )
# show(plot)


#### Embeddings des intitulées de catégories secondaires

In [ ]:
variable = "categories_fr"
df = pd.DataFrame(
    {"category": g.data[variable].apply(normalize_str).fillna("NA").unique()}
)
df_counts = g.data[variable].apply(normalize_str).fillna("NA").value_counts()
df["counts"] = [df_counts[v] for v in df["category"]]


threshold = (1, 10, 50, 100)
condlist = [df["counts"] <= n for n in threshold]
choicelist = [f"n <= {n}" for n in threshold] + [f"n > {threshold[-1]}"]
df["cat"] = pd.Categorical(
    np.select(condlist, choicelist[:-1], default="inf"), categories=choicelist
)

text_feature_to_2d_coord(
    data=df,
    feature_name="category",
    tfidf_kwargs=dict(min_df=2, max_df=0.5),
    tsne_kwargs=dict(perplexity=100, n_iter=2000),
    # use_umap=True,
    # tsne_kwargs=dict(n_neighbors=10, min_dist=0.1, spread=0.1),
    # force_compute=True,
)
plot = plot_tsne(
    df,
    "category",
    group_cat="cat",
    tooltip_var=["category", "counts", "cat"],
    interactive_plot=False,
)


In [ ]:
df = df.set_index("category")

g.data["categories_fr_emb_x"] = [
    df.loc[cat]["category_emb_x"]
    for cat in g.data["categories_fr"].apply(normalize_str).fillna("NA")
]
g.data["categories_fr_emb_y"] = [
    df.loc[cat]["category_emb_y"]
    for cat in g.data["categories_fr"].apply(normalize_str).fillna("NA")
]


In [ ]:
# plot = plot_tsne(
#     g.data,
#     "categories_fr",
#     group_cat="pnns_groups_1",
#     tooltip_var=["product_name", "nutrition_grade_fr", "categories_fr"],
# )
# show(plot)


## Exploration des noms de produit

In [ ]:
g.data["product_name"].sample(10)


### Embeddings des noms de produits

In [ ]:
text_feature_to_2d_coord(
    g.data,
    "product_name",
    tfidf_kwargs=dict(min_df=5, max_df=0.01),
    # use_umap=True,
    # tsne_kwargs=dict(n_neighbors=10, min_dist=1, spread=10, n_epochs=10000),
    # force_compute=True,
    # nb_samples=20000,
)


In [ ]:
plot = plot_tsne(
    g.data,
    "product_name",
    group_cat="pnns_groups_1",
    tooltip_var=["product_name", "nutrition_grade_fr"],
    interactive_plot=False,
)


In [ ]:
feature_summary(g.data)


## Exploration des noms de marques

In [ ]:
g.data["brands"].sample(10)


In [ ]:
df = (
    g.data["brands"].str.split(",").explode().apply(normalize_str).fillna("NA").unique()
)
df.shape


### Embeddings des noms de marques

In [ ]:
text_feature_to_2d_coord(
    g.data,
    "brands",
    tfidf_kwargs=dict(min_df=2, max_df=0.3),
    # use_umap=True,
    # tsne_kwargs=dict(n_neighbors=5, min_dist=1, spread=2),
    # force_compute=True,
    # nb_samples=1000,
)
plot = plot_tsne(
    g.data,
    "brands",
    group_cat="pnns_groups_1",
    tooltip_var=["product_name", "nutrition_grade_fr", "brands"],
    interactive_plot=False,
)


## Exploration des textes d'ingredients et des additifs

In [ ]:
g.data[["ingredients_text", "additives", "additives_n"]]


In [ ]:
df = g.data["ingredients_text"].dropna().apply(lambda s: normalize_str(s))
df = df.sample(2000)
df.sample(10)


In [ ]:
text_feature_to_2d_coord(
    g.data,
    "ingredients_text",
    tfidf_kwargs=dict(min_df=10, max_df=1.0),
    use_umap=True,
    tsne_kwargs=dict(min_dist=4, spread=4),
    # force_compute=True,
    # nb_samples=10000,
)
plot = plot_tsne(
    g.data,
    "ingredients_text",
    group_cat="pnns_groups_1",
    interactive_plot=False,
    tooltip_var=["product_name", "nutrition_grade_fr", "ingredients_text"],
)


Il est intéressant de voir que l'embedding (avec réduction de dimentionnalité) du texte des ingrédint permet de dégager des régions selon la catégorie de produits. Le recouvrement des points ne permet pas de dire exactement à quelle catégorie appartient un produit mais il est au moins possible d'identifier des catégories problbles auquelles appartient un produit. Nous pouvons donc affirmer que l'embedding "réduit" de texte des ingrédient contient encore de l'information pour caractériser les produits. 

In [ ]:
import re

regexp_finish_by_numbers = re.compile(".*-\d+")
regexp_number_only = re.compile("[a-z]+:\d+")


def clean_additives(raw_additives):
    if pd.isnull(raw_additives):
        return np.nan, np.nan, np.nan
    raw_additives = str(raw_additives)
    ingredients = raw_additives[3:-4].split("  ]  [ ")
    ingredients = list(
        filter(
            lambda s: not regexp_finish_by_numbers.match(s) or "-> exists" in s,
            ingredients,
        )
    )
    ingredients = [ing for ing in ingredients if ing != "en:fd-c"]
    additives = [s.split("->")[1].strip() for s in ingredients if "-> exists" in s]
    ingredients = [ing[ing.find(" -> ") + 4 :] for ing in ingredients]
    ingredients = list(
        filter(
            lambda s: not any(s in ing and s != ing for ing in ingredients), ingredients
        )
    )
    ingredients = list(filter(lambda s: not regexp_number_only.match(s), ingredients))
    return len(ingredients), ingredients, additives


g.data["additives_old"] = g.data["additives"]
results = g.data["additives_old"].map(clean_additives)
g.data["ingredients_n"] = results.map(lambda r: r[0])
g.data["ingredients"] = results.map(lambda r: r[1])
g.data["additives"] = results.map(lambda r: r[2])
g.data = g.data.drop(columns=["additives_old"])

g.data[["ingredients_n", "additives"]].sample(10)


# Exploration des quantités de sel et sodium

In [ ]:
g.data[["salt_100g", "sodium_100g"]].sample(10)


Les quantités de sel et de sodium suivent les même tendances. Regardons leur matrice de corrélations.

In [ ]:
g.data[["salt_100g", "sodium_100g"]].corr()


En effet, les deux variables sont complétements corrélées. Étudions leur rapport.

In [ ]:
(g.data["salt_100g"] / g.data["sodium_100g"]).sample(10)


Les deux variables sont liés par un rapport de 2,54. Il n'est donc pas utile de garder les deux variables. 
Nous décidons de garder "sodium_100g" car elle est celle utilisée pour le calcul du Nutri-Score [0].

[0] https://www.santepubliquefrance.fr/media/files/02-determinants-de-sante/nutrition-et-activite-physique/nutri-score/qr-scientifique-technique

In [ ]:
g.data = drop_features(g.data, "salt_100g")


# Exploration des variables nutriments

In [ ]:
nutriments = [
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "sodium_100g",
]
main_nutriments = [
    "fat_100g",
    "carbohydrates_100g",
    "fiber_100g",
    "proteins_100g",
    "sodium_100g",
]

g.data[nutriments].sample(10)


In [ ]:
feature_summary(g.data[nutriments])


Remplissage des quantités de nutriments manquantes. Si quelques variables de quantité de nutriments sont remplies pour un produit, on fait l'hypothèse que les quantités non-remplies sont égales à zéro, pour ce même produit.

In [ ]:
mask_nutriment_nulls = g.data[nutriments].isnull().to_numpy()
mask_nutriment_all_nulls = mask_nutriment_nulls.all(axis=1)
g.data.fillna({c: 0 for c in nutriments}, inplace=True)

g.data.loc[mask_nutriment_all_nulls, nutriments] = np.nan


In [ ]:
feature_summary(g.data[nutriments])


In [ ]:
g.data[nutriments].sample(10)


Quelques produits présentes des valeurs aberrantes avec des quantités de graisses inférieures aux quantités de graisses saturées, alors que les quantités de graisses saturées sont en toute logique incluses dans les quantitées de graisses. La même logique s'applique pour les glucides avec les sucres.
Nous allons donc corriger ces valeurs aberrantes, en considérant que les couples de valeurs sont égales.

In [ ]:
def assign_value_sub(data, main, sub):
    mask = (data[sub] > data[main]) & ~(data[main].isnull())
    print(f'Nb products with "{sub}" > "{main}": {mask.sum()}')
    data.loc[mask, main] = data[mask][sub]


assign_value_sub(g.data, main="fat_100g", sub="saturated-fat_100g")
assign_value_sub(g.data, main="carbohydrates_100g", sub="sugars_100g")
g.data[nutriments]


Regardons la distribution des variables nutritionnelles.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(25, 12))

df = g.data[["product_name"] + nutriments].copy()

df["total_100g"] = df[
    ["fat_100g", "carbohydrates_100g", "fiber_100g", "proteins_100g", "sodium_100g"]
].sum(axis=1)

axex = np.array(axes)
for col, ax in zip(nutriments + ["total_100g"], axes.flat):
    ax.set_title(col)
    bins = np.linspace(df[col].min(), df[col].max(), 100)
    df[col].plot.hist(bins=bins, ax=ax)
    df[col][(df[col] < 0) | (df[col] > 100)].plot.hist(bins=bins, ax=ax, color="red")
    ax.set_yscale("log")
    ax.set_xlabel("Quantité (g ou ml)")


Nous remarquons que certains produits présentes des valeurs aberrantes avec des quantitées, normalisées à 100g, suppérieures à 100. Ainsi qu'une proportion non néglégeable de produits qui présentent une somme total de quantités supérieure à 100g.

In [ ]:
mask = df["total_100g"] > 100
df[mask]


Pour ces produits dont la somme des quantités est supérieures à 100g, nous avons décidé mettre chacune de leur quantité de nutriments à nul, comme valeurs manquantes, pour faire du remplissage plus tard. 

In [ ]:
for nut in nutriments:
    mask = (g.data[nut] < 0) | (g.data[nut] > 100)
    g.data.loc[mask, nut] = pd.NA

mask = df["total_100g"] > 100
g.data.loc[mask, main_nutriments] = pd.NA


# Exploration de l'énergie (kJ/100g)

In [ ]:
df = g.data["energy_100g"]
df.describe()


Pour un produit, l'énergie maximale est estimée à un peu moins de 4000 KJ/100g, en considérant que ce produit est constitué uniquement de graisses.

In [ ]:
g.data[["product_name", "energy_100g"]].sort_values(by="energy_100g", ascending=False)[
    0:10
]


In [ ]:
energy_max = 4000
mask = df < energy_max
print(f"Nb de produits avec 'energy_100g' > 4000 kJ/100g : {mask.sum()}")
bins = np.linspace(df.min(), 10000, 200)
df[df < 10e3].plot.hist(bins=bins)
df[df > energy_max].plot.hist(bins=bins, color="red")
plt.yscale("log")
plt.xlabel("Energie (kJ/100g)")
_ = plt.ylabel("Nb produits (/bin)")


Nous remplaçons ces valeurs aberrantes par des valeurs nulles.

In [ ]:
g.data.loc[~mask, "energy_100g"] = np.nan


Nous pouvons estimer l'énergie colorique d'un produit à partir de ces quantités de nutriments, selon [0] & [1].

- [0] https://fr.wikipedia.org/wiki/Valeur_%C3%A9nerg%C3%A9tique
- [1] p.11, https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score/documents/rapport-2022-sur-les-modifications-de-l-algorithme-de-calcul-pour-les-aliments-solides-proposees-par-le-comite-scientifique-du-nutri-score

In [ ]:
def compute_energy(data):
    energy = {  # kJ/g
        "fat": 37,
        # "ethanol": 29,
        "proteins": 17,
        "carbohydrates": 17,
        "fiber": 8,
        "sodium": 0,
    }
    return pd.DataFrame(
        {comp: data[f"{comp}_100g"] * ener for comp, ener in energy.items()}
    ).sum(axis=1, skipna=False)


df = pd.DataFrame(
    {"computed_energy": compute_energy(g.data), "energy": g.data["energy_100g"]}
)


In [ ]:
((df["computed_energy"] - df["energy"]) / df["computed_energy"]).plot.hist(
    bins=np.linspace(-1, 1, 100)
)
plt.xlabel("Différence (%)")
_ = plt.ylabel("Nombre de produits/bin")


In [ ]:
_ = df["computed_energy"].plot.hist(bins=100)
plt.ylabel("Nb de produits (/bin)")
_ = plt.xlabel("Énergie calculée")


In [ ]:
null_energy = g.data["energy_100g"].isnull()
g.data.loc[null_energy, "energy_100g"] = compute_energy(g.data[null_energy])


On élimine les produits où le nutriscore n'est pas présent, puisque l'on veut constituer un dataset pour faire un entrainement supervisé de notre algorithme de prédiction de nutriscore.

In [ ]:
plt.figure(figsize=(14, 12))
col_order = [
    "product_name",
    "brands",
    "pnns_groups_1",
    "pnns_groups_2",
    "main_category_fr",
    "categories_fr",
    "ingredients_text",
    "ingredients_from_palm_oil_n",
    "ingredients_that_may_be_from_palm_oil_n",
    "additives_n",
    # "additives",
    "carbohydrates_100g",
    "fat_100g",
    "proteins_100g",
    "sodium_100g",
    "fiber_100g",
    "sugars_100g",
    "saturated-fat_100g",
    "energy_100g",
]
mat_corr = g.data[col_order].isnull().corr()

mask = np.zeros_like(mat_corr)
mask[np.triu_indices(mat_corr.shape[0])] = True
_ = sns.heatmap(
    mat_corr,
    annot=True,
    fmt=".0%",
    cmap="RdBu_r",
    vmin=-1,
    vmax=1,
    annot_kws={"fontsize": 9},
    linewidths=0.5,
    linecolor="gray",
)
sns.despine(
    fig=None,
    ax=None,
    top=False,
    right=False,
    left=False,
    bottom=False,
    offset=None,
    trim=False,
)
plt.show()


# Remplissage final des valeurs manquantes

In [ ]:
new_variable_order = [
    "code",
    "created_t",
    "product_name",
    "product_name_emb_x",
    "product_name_emb_y",
    "brands",
    "brands_emb_x",
    "brands_emb_y",
    "main_category_fr",
    "main_category_fr_emb_x",
    "main_category_fr_emb_y",
    "categories_fr",
    "categories_fr_emb_x",
    "categories_fr_emb_y",
    "pnns_groups_1",
    "pnns_groups_2",
    "ingredients_text",
    "ingredients_text_emb_x",
    "ingredients_text_emb_y",
    "ingredients",
    "ingredients_n",
    "additives",
    "additives_n",
    "ingredients_from_palm_oil_n",
    "ingredients_that_may_be_from_palm_oil_n",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "sodium_100g",
    "nutrition-score-fr_100g",
    "nutrition_grade_fr",
]

assert len(new_variable_order) == len(g.data.columns)

g.data = g.data[new_variable_order]

feature_summary(g.data)


In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [ ]:
numerical_features = list(g.data.select_dtypes(include=np.number).columns)
for f in ["nutrition-score-fr_100g"]:
    numerical_features.remove(f)
numerical_features


In [ ]:
pnns_transformer = OrdinalEncoder()
# pnns_transformer = OneHotEncoder(sparse_output=False)
X_pnns = pnns_transformer.fit_transform(g.data[["pnns_groups_1", "pnns_groups_2"]])
X_numerical = g.data[numerical_features].to_numpy()
X = np.concatenate([X_pnns, X_numerical], axis=1)

X_pnns.shape, X_numerical.shape, X.shape


In [ ]:
imputer = KNNImputer(n_neighbors=5)

res = imputer.fit_transform(X)
res.shape


In [ ]:
id_pnns = X_pnns.shape[1]
g.data.loc[:, ["pnns_groups_1", "pnns_groups_2"]] = pnns_transformer.inverse_transform(
    res[:, :id_pnns]
)
g.data["pnns_groups_1"] = g.data["pnns_groups_1"].astype("category")
g.data["pnns_groups_2"] = g.data["pnns_groups_2"].astype("category")
g.data.loc[:, numerical_features] = res[:, id_pnns:]


# Nettoyage des produits en doublon

In [ ]:
g.data.shape


In [ ]:
g.data["created_t"] = g.data["created_t"].astype("int64")

mask_dup = g.data.sort_values("created_t", ascending=True).duplicated(
    ["product_name", "brands"], keep="last"
)
print(mask_dup.size, g.data.shape)
mask_dup = mask_dup.sort_index()
g.data = g.data[~mask_dup]


In [ ]:
g.data.shape


# Sauvegarde des données nettoyées

In [ ]:
g.data.shape


In [ ]:
feature_summary(g.data)


In [ ]:
g.data.to_pickle("clean_data.pkl")
